In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [17]:
#load the trained models , scaler and onehotted code
model=load_model('model.h5')
#load the encoder and scaler
with open('label_encoder_class.pkl','rb') as file:
    label_encoder_class = pickle.load(file)
with open('label_encoder_ethnicity.pkl','rb') as file:
    label_encoder_ethnicity = pickle.load(file)
with open('label_encoder_fammem.pkl','rb') as file:
    label_encoder_fammem = pickle.load(file)
with open('label_encoder_jaundice.pkl','rb') as file:
    label_encoder_jaundice = pickle.load(file)
with open('label_encoder_sex.pkl','rb') as file:
    label_encoder_sex = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [31]:
# Example input data
input_data = {
    'A1': 1,
    'A2': 1,
    'A3': 0,
    'A4': 0,
    'A5': 0,
    'A6': 1,
    'A7': 1,
    'A8': 0,
    'A9': 0,
    'A10': 1,
    'Age_Mons': 36,
    'Qchat-10-Score': 4,
    'Sex': '1',
    'Ethnicity': 'White European',
    'Jaundice': 'yes',
    'Family_mem_with_ASD': 'no',
    'Class/ASD Traits ': 'Yes'
}


In [32]:
# One-hot encode 'Ethnicity'
ethnicity_encoded = label_encoder_ethnicity.transform([[input_data['Ethnicity']]]).toarray()
ethnicity_encoded_df = pd.DataFrame(ethnicity_encoded, columns=label_encoder_ethnicity.get_feature_names_out(['Ethnicity']))
ethnicity_encoded_df


d:\FinalYearProject\main\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Ethnicity_Hispanic,Ethnicity_Latino,Ethnicity_Native Indian,Ethnicity_Others,Ethnicity_Pacifica,Ethnicity_White European,Ethnicity_asian,Ethnicity_black,Ethnicity_middle eastern,Ethnicity_mixed,Ethnicity_south asian
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
input_df=pd.DataFrame([input_data])
input_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Class/ASD Traits
0,1,1,0,0,0,1,1,0,0,1,36,4,1,White European,yes,no,Yes


In [34]:
## Encode categorical variables
input_df['Class/ASD Traits ']=label_encoder_class.fit(input_df['Class/ASD Traits '])


In [35]:

input_df['Family_mem_with_ASD']=label_encoder_fammem.transform(input_df['Family_mem_with_ASD'])
input_df['Jaundice']=label_encoder_jaundice.transform(input_df['Jaundice'])
# input_df['Sex']=label_encoder_sex.transform(input_df['Sex'])
input_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Class/ASD Traits
0,1,1,0,0,0,1,1,0,0,1,36,4,1,White European,1,0,LabelEncoder()


In [36]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Ethnicity",axis=1),ethnicity_encoded_df],axis=1)


In [37]:
# Drop columns not used during training (e.g., target column 'Class/ASD Traits ')
if 'Class/ASD Traits ' in input_df.columns:
    input_df = input_df.drop(columns=['Class/ASD Traits '])

# Reindex the input DataFrame to match the scaler's feature names
aligned_input_df = input_df.reindex(columns=scaler.feature_names_in_, fill_value=0)

# Scale the aligned input
input_scaled = scaler.transform(aligned_input_df)

print("Scaled Input:", input_scaled)


Scaled Input: [[ 0.86962636  1.10646934 -0.80804776 -1.04241722 -1.06497757  0.84876862
   0.71656472 -0.94346941 -0.99644759  0.82621712  1.00885301 -0.44127304
   0.66224358  1.63254935 -0.43473968 -0.18190172 -0.1711842  -0.05976143
  -0.19209469 -0.07724369  1.43716893 -0.61986809 -0.24024245 -0.45388168
  -0.07724369 -0.25110098]]


In [38]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 0.86962636,  1.10646934, -0.80804776, -1.04241722, -1.06497757,
         0.84876862,  0.71656472, -0.94346941, -0.99644759,  0.82621712,
         1.00885301, -0.44127304,  0.66224358,  1.63254935, -0.43473968,
        -0.18190172, -0.1711842 , -0.05976143, -0.19209469, -0.07724369,
         1.43716893, -0.61986809, -0.24024245, -0.45388168, -0.07724369,
        -0.25110098]])

In [39]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 21ms/step


array([[1.]], dtype=float32)

In [40]:
prediction_proba = prediction[0][0]

In [41]:
prediction_proba

1.0

In [42]:
if prediction_proba > 0.5:
    print('The customer is likely to asd.')
else:
    print('The customer is not likely to asd.')

The customer is likely to asd.
